In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

def load_and_validate_data():
    print("Loading and validating data...")
    customers = pd.read_csv('/content/Customers.csv')
    transactions = pd.read_csv('/content/Transactions.csv')
    customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])
    transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])
    return customers, transactions

def create_customer_features(customers, transactions):
    print("\nGenerating customer features...")
    latest_date = transactions['TransactionDate'].max()
    customers['AccountAge'] = (latest_date - customers['SignupDate']).dt.days
    customer_metrics = transactions.groupby('CustomerID').agg({
        'TransactionID': 'count',
        'TransactionDate': lambda x: (latest_date - x.max()).days,
    }).rename(columns={'TransactionID': 'FrequencyScore', 'TransactionDate': 'RecencyScore'})
    monetary = transactions.groupby('CustomerID').size().reset_index(name='MonetaryScore')
    features = (customers[['CustomerID', 'Region', 'AccountAge']]
               .merge(customer_metrics, on='CustomerID')
               .merge(monetary, on='CustomerID'))
    return features

def prepare_clustering_data(features):
    numerical_cols = ['AccountAge', 'FrequencyScore', 'RecencyScore', 'MonetaryScore']
    region_dummies = pd.get_dummies(features['Region'], prefix='Region')
    X = pd.concat([features[numerical_cols], region_dummies], axis=1)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, X.columns

def find_optimal_clusters(X_scaled, max_clusters=10):
    print("\nFinding optimal number of clusters...")
    db_scores, inertia_scores, silhouette_scores = [], [], []
    k_range = range(2, max_clusters + 1)
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X_scaled)
        db_scores.append(davies_bouldin_score(X_scaled, labels))
        inertia_scores.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X_scaled, labels))
        print(f"K={k}, DB Index: {db_scores[-1]:.4f}, Silhouette Score: {silhouette_scores[-1]:.4f}, Inertia: {inertia_scores[-1]:.4f}")
    optimal_k = k_range[np.argmin(db_scores)]
    return optimal_k, min(db_scores), silhouette_scores[np.argmin(db_scores)], inertia_scores[np.argmin(db_scores)]

def perform_clustering(X_scaled, features, optimal_k):
    print(f"\nPerforming final clustering with {optimal_k} clusters...")
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    features['Cluster'] = kmeans.fit_predict(X_scaled)
    cluster_profiles = features.groupby('Cluster').agg({
        'AccountAge': 'mean',
        'FrequencyScore': 'mean',
        'RecencyScore': 'mean',
        'MonetaryScore': 'mean'
    }).round(2)
    region_dist = pd.crosstab(features['Cluster'], features['Region'], normalize='index').round(2)
    avg_cluster_size = len(features) / optimal_k
    return features, cluster_profiles, region_dist, avg_cluster_size

def main():
    customers, transactions = load_and_validate_data()
    features = create_customer_features(customers, transactions)
    X_scaled, feature_names = prepare_clustering_data(features)
    optimal_k, best_db_score, best_silhouette_score, best_inertia = find_optimal_clusters(X_scaled)
    results, profiles, region_dist, avg_cluster_size = perform_clustering(X_scaled, features, optimal_k)
    print("\nClustering Analysis Report")
    print("=" * 50)
    print(f"Number of clusters: {optimal_k}")
    print(f"Davies-Bouldin Index: {best_db_score:.4f}")
    print(f"Silhouette Score: {best_silhouette_score:.4f}")
    print(f"Inertia (Sum of Squared Distances): {best_inertia:.4f}")
    print(f"Average Cluster Size: {avg_cluster_size:.2f}")
    print("\nCluster Profiles:")
    print(profiles)
    print("\nRegional Distribution:")
    print(region_dist)
    results[['CustomerID', 'Cluster']].to_csv('cluster_assignments.csv', index=False)
    profiles.to_csv('cluster_profiles.csv')
    region_dist.to_csv('regional_distribution.csv')

if __name__ == "__main__":
    main()


Loading and validating data...

Generating customer features...

Finding optimal number of clusters...
K=2, DB Index: 2.0794, Silhouette Score: 0.1530, Inertia: 1319.7011
K=3, DB Index: 1.4797, Silhouette Score: 0.2953, Inertia: 1050.9831
K=4, DB Index: 1.3778, Silhouette Score: 0.3141, Inertia: 883.9486
K=5, DB Index: 1.0623, Silhouette Score: 0.3811, Inertia: 686.0516
K=6, DB Index: 0.9982, Silhouette Score: 0.3977, Inertia: 580.8527
K=7, DB Index: 1.0782, Silhouette Score: 0.3503, Inertia: 528.6378
K=8, DB Index: 1.0957, Silhouette Score: 0.3218, Inertia: 483.5911
K=9, DB Index: 1.1266, Silhouette Score: 0.3146, Inertia: 422.5907
K=10, DB Index: 1.1257, Silhouette Score: 0.3049, Inertia: 403.5996

Performing final clustering with 6 clusters...

Clustering Analysis Report
Number of clusters: 6
Davies-Bouldin Index: 0.9982
Silhouette Score: 0.3977
Inertia (Sum of Squared Distances): 580.8527
Average Cluster Size: 33.17

Cluster Profiles:
         AccountAge  FrequencyScore  RecencySco